## Задание на собеседование

Имеются две базы данных (два листа Excel-файла): база с ценами конкурентов (`Data_Parsing`) и внутренняя база компании (`Data_Company`).

В базе парсинга есть два `id`, однозначно определяющие товар: `producer_id` и `producer_color`.

В базе компании есть два аналогичных поля: `item_id` и `color_id`.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: `_, -, ~, \\, /`.

Необходимо:

- Считать данные из Excel в DataFrame (`Data_Parsing``) и (`Data_Company``).
- Подтянуть к базе парсинга данные из базы компании (`item_id`, `color_id`, `current_price`) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).
- Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.
- Записать новый файл `Excel` с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец `outlier` и проставить `Yes` для выбросов).

In [39]:
import pandas as pd
import plotly.express as px

In [3]:
data_parsing = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Parsing')
display(data_parsing.head())

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa1111_11,black,167
1,Valentino,Shoes,aaaaa1111_12,black,188
2,Valentino,Shoes,aaaaa1111_13,black,184
3,Valentino,Shoes,aaaaa1111_14,bla//ck,196
4,Valentino,Shoes,aaaaa1111_15,bla\\ck,250


In [9]:
data_parsing['producer_color'].value_counts()

producer_color
~~~red      33
white_~     21
black        9
bla//ck      3
bla\\ck      3
bla__ck      3
bla\\c~k     3
Name: count, dtype: int64

In [ ]:
for ind, color in enumerate(data_parsing['producer_color'].values):
    if 'red' in color:
        data_parsing['producer_color'].loc[ind] = 'red'
    elif 'white' in color:
        data_parsing['producer_color'].loc[ind] = 'white'
    elif 'bla' in color:
        data_parsing['producer_color'].loc[ind] = 'black'

In [23]:
data_parsing['producer_color'].value_counts()

producer_color
red      33
black    21
white    21
Name: count, dtype: int64

In [ ]:
for ind, id in enumerate(data_parsing['producer_id'].values):
    data_parsing['producer_id'].loc[ind] = id.replace('_', '')

In [28]:
data_parsing = data_parsing.rename(columns={'producer_id':'item_id', 'producer_color':'color_id'})

In [29]:
display(data_parsing.head())

,brand,Category,item_id,color_id,price
0,Valentino,Shoes,aaaaa111111,black,167
1,Valentino,Shoes,aaaaa111112,black,188
2,Valentino,Shoes,aaaaa111113,black,184
3,Valentino,Shoes,aaaaa111114,black,196
4,Valentino,Shoes,aaaaa111115,black,250


In [4]:
data_company = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Company')
display(data_company.head())

,brand,Category,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,153


In [35]:
data_total = data_parsing.merge(data_company, on=['brand', 'Category', 'item_id', 'color_id'], how='left')
display(data_total)

,brand,Category,item_id,color_id,price,current price
0,Valentino,Shoes,aaaaa111111,black,167,247
1,Valentino,Shoes,aaaaa111112,black,188,161
2,Valentino,Shoes,aaaaa111113,black,184,234
3,Valentino,Shoes,aaaaa111114,black,196,167
4,Valentino,Shoes,aaaaa111115,black,250,153
...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,165
71,Stone Island,Bags,sssss111132,red,158,196
72,Stone Island,Bags,sssss111133,red,194,236
73,Stone Island,Bags,sssss111134,red,256,222


In [36]:
data_total['price_difference'] = round(data_total['price'] / data_total['current price'] * 100, 2)
display(data_total.head())

,brand,Category,item_id,color_id,price,current price,price_difference
0,Valentino,Shoes,aaaaa111111,black,167,247,67.61
1,Valentino,Shoes,aaaaa111112,black,188,161,116.77
2,Valentino,Shoes,aaaaa111113,black,184,234,78.63
3,Valentino,Shoes,aaaaa111114,black,196,167,117.37
4,Valentino,Shoes,aaaaa111115,black,250,153,163.40


In [37]:
data_total = data_total.rename(columns={'Category':'category', 'price':'competitors_price', 'current price':'company_price'})
display(data_total.head())

,brand,category,item_id,color_id,competitors_price,company_price,price_difference
0,Valentino,Shoes,aaaaa111111,black,167,247,67.61
1,Valentino,Shoes,aaaaa111112,black,188,161,116.77
2,Valentino,Shoes,aaaaa111113,black,184,234,78.63
3,Valentino,Shoes,aaaaa111114,black,196,167,117.37
4,Valentino,Shoes,aaaaa111115,black,250,153,163.40


In [38]:
data_total[['brand', 'competitors_price', 'company_price', 'price_difference']].groupby(by='brand').mean()

,competitors_price,company_price,price_difference
brand,,,
Brioni,222.64,218.60,104.7528
Stone Island,218.80,221.84,101.8688
Valentino,217.96,226.76,101.3584


In [40]:
fig = px.box(
    data_frame=data_total,
    x='price_difference',
    y='brand'
)
fig.show()

In [41]:
data_total.to_excel('data/data_total.xlsx')